# Connect to Neo4j 
using Neo4j Bolt Driver

In [62]:
#first install neo4j driver
!pip install neo4j-driver

In [63]:
from neo4j import GraphDatabase

In [64]:
# instantiate driver object
driver = GraphDatabase.driver("bolt://localhost:7687", auth=("neo4j", "neo4j"))

In [65]:
# Checking number of nodes in the database
with driver.session() as session:
    result = session.run("MATCH (a) RETURN COUNT(a) AS num")
    
    for record in result:
        print("Number of nodes in the database: %i" % record['num'])

Number of nodes in the database: 4044069


## CREATE

### Create Characters



In [66]:
# 1. Creating constraint on character node to property: name
with driver.session() as session:
    session.run("CREATE CONSTRAINT ON (c:Character) ASSERT c.name IS UNIQUE;")

In [67]:
# Query to create Character node
# Parameterised cypher query
create_character_query = '''
MERGE (c:Character {name: $name})
'''

In [46]:
with driver.session() as session:
    with open("data/characters.txt") as f:
        for line in f:
            # strip whitespace
            text = line.strip()
            # execute create_character_query, passing name parameter
            session.run(create_character_query, parameters={'name':text})
        

### Create Line nodes

> For each line in the text, CREATE a corresponding record in the database. Each record will include the name of the character speaking, the (absolute) line number of the phrase and the phrase itself, trimmed of any leading or following spaces

In [68]:
# Query to add a line to the database
create_line_query = '''
CREATE (l:Line {text: $line_text, number: $line_num})
WITH l
MATCH (c:Character) WHERE c.name = $current_character
CREATE (l)<-[:SPEAKS]-(c)
'''


In [49]:
# create an empty set
characters = set()

# fetch all Character names from the database
get_characters_query = '''
MATCH (c:Character)
RETURN c.name AS name
'''

# for each Character name, add it to the characters set
with driver.session() as session:
    result = session.run(get_characters_query)
    for record in result:
        characters.add(record['name'])

characters

{'BOTTOM',
 'COBWEB',
 'DEMETRIUS',
 'EGEUS',
 'FLUTE',
 'HELENA',
 'HERMIA',
 'HIPPOLYTA',
 'LYSANDER',
 'MOTH',
 'MUSTARDSEED',
 'OBERON',
 'PEASEBLOSSOM',
 'PHILOSTRATE',
 'PUCK',
 'QUINCE',
 'SNOUT',
 'SNUG',
 'STARVELING',
 'THESEUS',
 'TITANIA'}

In [69]:
# import the time 
import time

In [70]:
line_num = 0

begin = time.time()
with driver.session() as session:
    with open("data/A_Midsummer_Nights_Dream.txt") as f:
        current_character = ''
        for line in f:
            # strip leading and trailing whitespace
            text = line.strip()
            # is line the character speaking?
            if text in characters:
                current_character = text
            else:
                line_num += 1
                session.run(create_line_query, parameters={
                    'line_text': text,
                    'line_num': line_num,
                    'current_character': current_character
                })
            # insert line into database
            
    end = time.time()
    create_time = (end-begin)/line_num * 1000

print(str(create_time))
    

0.4032414976362985


## UPDATE

> For each record in the database, search for character names, convert them to UPPERCASE, then UPDATE the record in the database

In [71]:
# query to update 
update_query = '''
WITH $character AS character
WITH substring(character,0,1) + toLower(substring(character,1)) AS casedChar, character

MATCH (l:Line) WHERE toLower(l.text) CONTAINS toLower(character)
SET l.text = replace(l.text, casedChar, character)
RETURN COUNT(l) AS num
'''

In [72]:
update_count = 0

with driver.session() as session:
    begin = time.time()
    for c in characters:
        # Update the lines mentioning this character
        result = session.run(update_query, parameters = {'character': c})
        # increment update_count
        record = result.single()
        update_count += record['num']
    end = time.time()
    update_time = (end-begin)/update_count * 1000
    print(update_time)

0.36825304446013074


## DELETE

> For each record in the database, the instructor will also need to DELETE any lines that start with “ENTER” , “EXIT” , “ACT” , or “SCENE”


In [73]:
# query to delete stop words
delete_query = '''
WITH ['ENTER', 'EXIT', 'ACT', 'SCENE'] AS words
MATCH (l:Line) WHERE any(x IN words WHERE toLower(l.text) STARTS WITH toLower(x))
DETACH DELETE l
RETURN COUNT(l) AS num
'''

In [74]:
with driver.session() as session:
    begin = time.time()
    result = session.run(delete_query)
    record = result.single()
    num_lines = record['num']
    end = time.time()
    delete_time = (end-begin) / num_lines * 1000
    print(delete_time)

0.37057225297137003


## READ

When all other tasks are complete, the next step will be to READ each line and print it out to console


In [75]:
# query to read lines
read_lines_query = '''
MATCH (l:Line) RETURN l.text AS text, l.number AS linenum
'''

In [79]:
with driver.session() as session:
    begin = time.time()
    # read each line from the datbase
    result = session.run(read_lines_query)
    #for record in result:
    # print each line
        # print("%i: %s" % (record['linenum'], record['text']))
    end = time.time()
    
    # get number of lines in database
    result = session.run("MATCH (l:Line) RETURN COUNT(l) AS num")
    numlines = result.single()['num']
    
    
    
    read_time = (end-begin)/numlines * 1000
    print(str(read_time))

0.0006957822551264142


In [80]:
print("CREATE: %.2fms per line" % create_time)
print("READ: %.2fms per line" % read_time)
print("UPDATE %.2fms per line" % update_time)
print("DELETE %.2fms per line" % delete_time)

CREATE: 0.40ms per line
READ: 0.00ms per line
UPDATE 0.37ms per line
DELETE 0.37ms per line
